# Limpieza 2: Strings

### Importamos las librerías.

In [210]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [211]:
# Abrimos la BBDD

df = pd.read_csv('datos/Limpieza-1.csv', index_col=0)
df.head(3)

,year,type,country,activity,age,species,month,fatal,sex
case_number,,,,,,,,,
2018.06.25,2018,Boating,usa,Paddling,57,White shark,Jun,N,F
2018.06.03.a,2018,Unprovoked,brazil,Swimming,18,Tiger shark,Jun,Y,M
2018.05.26.b,2018,Unprovoked,usa,Walking,15,"Bull shark, 6'",May,N,M


In [212]:
# Realizamos una copia de seguridad

df1 = df.copy()

#### 1. Columna de species: 

Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull).

El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified".

In [213]:
# Creamos una función para detectar todas aquellas coincidencias de las categorías del enunciado mediante regex
# Renombramos los valores de la columna, para faciliar su legiblidad
# Añadimos un try, except para evitar posibles errores con los valores nulos

def clasificar(x):
    
    patron_blanco = r".*[Ww](hite|HITE).*"
    patron_tiger = r".*[Tt](iger|IGER).*"
    patron_gris = r".*[Gg](rey|REY).*"
    patron_toro = r".*[Bb](ull|ULL).*"
    patron_lemon = r".*[Ll](emon|EMON).*"
    
    try:
        if re.findall(patron_blanco, x):
            return x.replace(x, "White Shark")
        elif re.findall(patron_tiger, x):
            return x.replace(x, "Tiger Shark")
        elif re.findall(patron_gris, x):
            return x.replace(x, "Grey Shark")
        elif re.findall(patron_toro, x):
            return x.replace(x, "Bull Shark")
        elif re.findall(patron_lemon, x):
            return x.replace(x, "Lemon Shark")

        else:
            return x.replace(x, "Unspecified")

    except: 
        return np.nan

In [214]:
# Aplicamos la función

df['species'] = df['species'].apply(clasificar)

In [215]:
# Comprobamos los valores únicos

df['species'].unique()

array(['White Shark', 'Tiger Shark', 'Bull Shark', 'Grey Shark',
       'Unspecified', 'Lemon Shark', nan], dtype=object)

##### 2. Columna de age: 

es una columna de tipo string pero debería ser de tipo integer. 

Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?

Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.
Por último cambiad el tipo de la columna de string a integer.

In [216]:
# Cambiamos el tipo de los valores de la columna para poder trabajar con ella

df['age'] = df['age'].astype(str)

In [217]:
# Comprobamos los valores únicos para ver qué patrón de regex debemos utilizar

df['age'].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '60s',
       '51', '61', '59', '42', '6', '27', '23', '29', '39', '24', '12',
       '26', '71', '43', '9', '44', '14', '62', '52', '38', '68', '16',
       '47', '63', '70', '41', '40s', '53', '20s', '7', '66', '45', '74',
       '64', '8', '56', '49', '18 or 20', 'Teen', '30s', '77', '8 or 10',
       '84', '\xa0 ', ' ', '30 or 36', '6½', '5', ' 30', ' 28', "60's",
       '67', '>50', '? & 19', '21, 34,24 & 35', '30 & 32', '13 or 18',
       '7 or 8', '9 or 10', 'nan', 'young', '13 or 14'], dtype=object)

In [219]:
# Creamos una función para rescatar sólo los números de los valores

def edad(col):
    patron_edad = "\d+"
    try:
        return re.findall(patron_edad, col)[0]
    except:
        return np.nan

In [220]:
# Aplicamos la función

df['age'] = df['age'].apply(edad)

In [221]:
# Comprobamos los valores únicos

df["age"].unique()

array(['57', '18', '15', '32', '21', '30', '60', '33', '19', '25', '10',
       '69', '55', '35', '20', '54', '22', '31', '40', '37', '11', '13',
       '34', '50', '46', '48', '17', '28', '65', '73', '58', '36', '51',
       '61', '59', '42', '6', '27', '23', '29', '39', '24', '12', '26',
       '71', '43', '9', '44', '14', '62', '52', '38', '68', '16', '47',
       '63', '70', '41', '53', '7', '66', '45', '74', '64', '8', '56',
       '49', nan, '77', '84', '5', '67'], dtype=object)

In [222]:
# Volvemos a convertir el tipo de los valores a integers

df['age'] = df['age'].astype("int64", errors = 'ignore')

- Optamos por dar cómo nulos aquellas filas con más de un valor, ya que sin más información, no sabemos qué valor es el correcto.

#### 3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [224]:
df.to_csv("datos/Limpieza-2.csv", index = False)